In [1]:
import langchain  
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain, ConversationalRetrievalChain, StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
import llama_index
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
import textwrap

from rouge_score import rouge_scorer
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA


from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader("/Users/anantsinha/Downloads/7df4dbdc-eb62-4d53-bc27-d334bfcb2335-2.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 20}
)

query = "How many Lawsuits were filled?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

Further, in connection with the conflict, a significant number and percentage of our employees have
39
----------------------------------------------------------------------------------------------------
Document 2:

$3.72 billion and $7.01 billion, respectively. During the third quarter and first nine months of fiscal year 2023, we repurchased 28 million and
56 million shares of our common stock for
21
----------------------------------------------------------------------------------------------------
Document 3:

The following table presents details of our share repurchase transactions during the third quarter of fiscal year 2024:
45
----------------------------------------------------------------------------------------------------
Document 4:

the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for
the past 90 days. Yes B3] No
Indicate by check mark whether the 

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke("How many lawsuits were filled?")
pretty_print_docs(compressed_docs)

Document 1:

Further, in connection with the conflict, a significant number and percentage of our employees have
39
----------------------------------------------------------------------------------------------------
Document 2:

complaint without leave to amend, entered judgment in favor of NVIDIA and closed the case. On March 30, 2021, plaintiffs filed an appeal from
judgment
20
$
$
6,499
11,861
1,128
1,038
660
354
21,540
Commitments
(In millions)
----------------------------------------------------------------------------------------------------
Document 3:

the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for
the past 90 days. Yes B3] No
Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during
